# Supervised Fine tuning GPT 


In [1]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}

{'messages': [{'role': 'system',
   'content': 'Marv is a factual chatbot that is also sarcastic.'},
  {'role': 'user', 'content': "What's the capital of France?"},
  {'role': 'assistant',
   'content': "Paris, as if everyone doesn't know that already."}]}

In [7]:
import pandas as pd
import json
import re
from sklearn.metrics import classification_report


In [3]:
train_path = '../Data/team_train.json'
with open(train_path, 'r', encoding='utf-8') as file:
    data_train = json.load(file)

dev_path = '../Data/team_dev.json'
with open(dev_path, 'r', encoding='utf-8') as file:
    data_dev = json.load(file)

test_path = '../Data/team_test.json'
with open(test_path, 'r', encoding='utf-8') as file:
    data_test = json.load(file)

## Data Preprocessing

In [5]:
def clean_text(text):
    # 去除多余的空格和换行符，并在原始有换行符的地方加上句号
    cleaned_text = re.sub(r'\s*\n+\s*', '。', text)
    # 去除多余的空格
    cleaned_text = re.sub(r'\s+', '', cleaned_text)
    return cleaned_text

## Fine-tuning GPT

In [ ]:
def transform_data(data, output_path):
    transformed_data = []
    for ID, sentence1, sentence2, label in data:
        sentence1 = clean_text(sentence1)
        sentene2 = clean_text(sentence2)
        transformed_data.append({
            "messages": [
                {"role": "system", "content": "識別社交媒體討論線程中的攻擊和支持論證關係。每個實例包含兩條帖子，並且該數據集中有三個標籤：支持（輸出'1'）、攻擊（輸出'2'）和無關係（輸出'0'）。目標是確定第二條帖子是否支持/攻擊/無關係第一條帖子。來源是社交媒體（mobile01），語言是繁體中文。若答案是支持，則輸出數字1, 若反對則輸出2，若無關係則輸出0"},
                {"role": "user", "content": '第一句：'+sentence1+'\n 第二句：'+ sentence2},
                {"role": "assistant", "content": str(label)}
            ]
        })
    with open(output_path, 'w', encoding='utf-8') as file:
        for entry in transformed_data:
            file.write(json.dumps(entry, ensure_ascii=False) + "\n")

# Paths for the transformed data
transformed_train_path = '../Data/Task1_transformed_train.jsonl'
transformed_dev_path = '../Data/Task1_transformed_dev.jsonl'

with open('../Data/team_train.json', 'r') as file:
    train_data = json.load(file)

with open('../Data/team_dev.json', 'r') as file:
    dev_data = json.load(file)
# Transform the datasets
transform_data(train_data, transformed_train_path)
transform_data(dev_data, transformed_dev_path)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key= 'your OpenAI API Key')

training_file = client.files.create(
    file=open(transformed_train_path, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(transformed_dev_path, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)
     

Training file id: file-cbYExMlXBvrU9Lstq24Rqvxt
Validation file id: file-RJCVUIZIFvqR7bt3uM12iClN


In [49]:
suffix_name = "task1_classifier"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

FineTuningJob(id='ftjob-jiytJk6W4qHbPBfGoN820w6A', created_at=1717128103, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bbOoKTcdENP64l8faECYvNSy', result_files=[], status='validating_files', trained_tokens=None, training_file='file-cbYExMlXBvrU9Lstq24Rqvxt', validation_file='file-RJCVUIZIFvqR7bt3uM12iClN', user_provided_suffix='task1_classifier', seed=344356249, estimated_finish=None, integrations=[])

In [ ]:
jobID = response.id
client.fine_tuning.jobs.list_events(fine_tuning_job_id= jobID, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-CeXddHFZgCL64pD0DHZdtwAC', created_at=1717128021, level='error', message='Creating this fine-tuning job would exceed your hard limit, please check your plan and billing details.                     Cost of job ftjob-CzQnhWimOqFZddHI9ZRgfDMQ: USD 84.38. Quota remaining for org-bbOoKTcdENP64l8faECYvNSy: USD 18.81.', object='fine_tuning.job.event', data={'error_code': 'exceeded_quota'}, type='message'), FineTuningJobEvent(id='ftevent-RoFKc1UoYgr8y0Yb53XpRSF5', created_at=1717127934, level='info', message='Validating training file: file-cbYExMlXBvrU9Lstq24Rqvxt and validation file: file-RJCVUIZIFvqR7bt3uM12iClN', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-soVM3lM6KWQvmGYFVHjkH26c', created_at=1717127934, level='info', message='Created fine-tuning job: ftjob-CzQnhWimOqFZddHI9ZRgfDMQ', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [ ]:
response = client.fine_tuning.jobs.retrieve(jobID)
response

FineTuningJob(id='ftjob-pdU69tixxycisyPS1mvmY6zn', created_at=1717060602, error=Error(code='exceeded_quota', message='Creating this fine-tuning job would exceed your hard limit, please check your plan and billing details.                     Cost of job ftjob-pdU69tixxycisyPS1mvmY6zn: USD 84.38. Quota remaining for org-bbOoKTcdENP64l8faECYvNSy: USD 18.81.', param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=13, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-bbOoKTcdENP64l8faECYvNSy', result_files=[], status='failed', trained_tokens=None, training_file='file-NpJjvcJQHV570kfk1NX0WX5A', validation_file=None, user_provided_suffix='task1_classifier', seed=250590840, estimated_finish=None, integrations=[])

In [48]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: None


In [ ]:
results = pd.DataFrame()
index = []
sent1 = []
sent2 = []
#observed = []
pred = []
for ID, sentence1, sentence2 in data_test:
    index.append(ID)
    sent1.append(clean_text(sentence1))
    sent2.append(clean_text(sentence2))
    #observed.append(int(label))
    completion = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=[
            {"role": "system", "content": "This is a classification task. Classify the following sentence as 'premise' than output '0' or 'claim' than output '1'."},
            {"role": "user", "content": '第一句：'+sentence1+'\n 第二句：'+ sentence2}
            ]
        )
    if completion.choices[0].message.content == 'premise':
        pred.append(0)
    else:
        pred.append(1)

results['ID'] = index
results['y_pred'] = pred

# sava as .csv and submit to Kaggle
results.to_csv('../Result/GPTfinetune_results.csv', index=None)
